In [ ]:
from pathlib import Path

import albumentations as A
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torchvision.utils
from albumentations.pytorch import ToTensorV2
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms.functional import normalize

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
img_dir = "original/"
mask_dir = "annotated/"

img_paths = [file for file in Path(img_dir).iterdir() if not file.name.startswith(".")]
mask_paths = [
    file for file in Path(mask_dir).iterdir() if not file.name.startswith(".")
]

In [ ]:
df = pd.DataFrame({"image_path": img_paths, "mask_paths": mask_paths}, dtype=str)

In [ ]:
class CuffDataset(Dataset):
    def __init__(self, df, transforms):
        # df contains the paths to all files
        self.df = df
        # transforms is the set of data augmentation operations
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image = np.array(Image.open(self.df.iloc[idx, 0]), dtype=np.float32) / 255
        mask = np.array(Image.open(self.df.iloc[idx, 1]), dtype=np.float32) / 255

        augmented = self.transforms(image=image, mask=mask)
        image = augmented["image"]  # Dimension (3, 255, 255)
        mask = augmented["mask"]  # Dimension (255, 255)
        image = normalize(
            image, mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5), inplace=True
        )
        mask = np.expand_dims(mask, axis=0)
        return image, mask

In [ ]:
PATCH_SIZE = 256

transforms = A.Compose(
    [
        A.RandomCrop(width=PATCH_SIZE, height=PATCH_SIZE, p=1.0),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.Transpose(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.01, scale_limit=0.04, rotate_limit=0, p=0.25),
        ToTensorV2(),
    ]
)

In [ ]:
# Split df into train and test data
train_df, val_df = train_test_split(df, test_size=0.2)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [ ]:
train_dataset = CuffDataset(train_df, transforms=transforms)
train_dataloader = DataLoader(train_dataset, batch_size=28, shuffle=False)

val_dataset = CuffDataset(val_df, transforms=transforms)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [ ]:
def imshow(img):
    npimg = np.float32(img)
    npimg = np.transpose(npimg, (1, 2, 0))
    plt.imshow(npimg, vmin=0, vmax=1)


dataiter = iter(train_dataloader)
images, masks = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
plt.figure()
imshow(torchvision.utils.make_grid(masks))
plt.figure()